# Initializing WandB project and Reading The Data

In [1]:
# import necessary libraries
import wandb
import pandas as pd

# initialize wandb project, group the runs under the "eda"
run = wandb.init(project="nyc_airbnb", group="eda", save_code=True)
# download the data from the artifact store
local_path = wandb.use_artifact("alturkim/nyc_airbnb/sample.csv:latest").file()
# read the downloaded data
df = pd.read_csv(local_path)

wandb: Currently logged in as: alturkim. Use `wandb login --relogin` to force relogin


# Generate a ydata_profiling profile
We start by generating an automatic report using ydata_profiling, this helps deciding the necessary cleaning and preprocessing steps to be done later.

In [2]:
import ydata_profiling

profile = ydata_profiling.ProfileReport(df)
profile.to_widgets()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

,id,host_id,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,neighbourhood_group,room_type
id,1.000000,0.558918,0.004996,0.070697,-0.020739,-0.057798,-0.307933,0.360443,0.135250,0.165714,0.064032,0.070261
host_id,0.558918,1.000000,0.049621,0.108965,-0.071772,-0.130251,-0.128008,0.267899,0.147273,0.173292,0.100284,0.092129
latitude,0.004996,0.049621,1.000000,0.035107,0.136172,0.022289,-0.044354,-0.023149,0.003526,-0.006962,0.539251,0.117330
longitude,0.070697,0.108965,0.035107,1.000000,-0.438035,-0.118921,0.079847,0.119089,0.063523,0.068899,0.654091,0.157505
price,-0.020739,-0.071772,0.136172,-0.438035,1.000000,0.101289,-0.055009,-0.018527,-0.106083,0.086088,0.018278,0.024730
minimum_nights,-0.057798,-0.130251,0.022289,-0.118921,0.101289,1.000000,-0.174905,-0.288797,0.064117,0.075981,0.002962,0.011713
number_of_reviews,-0.307933,-0.128008,-0.044354,0.079847,-0.055009,-0.174905,1.000000,0.706208,0.056054,0.236664,0.027393,0.021426
reviews_per_month,0.360443,0.267899,-0.023149,0.119089,-0.018527,-0.288797,0.706208,1.000000,0.146484,0.392126,0.047511,0.028754
calculated_host_listings_count,0.135250,0.147273,0.003526,0.063523,-0.106083,0.064117,0.056054,0.146484,1.000000,0.406606,0.089015,0.097468
availability_365,0.165714,0.173292,-0.006962,0.068899,0.086088,0.075981,0.236664,0.392126,0.406606,1.000000,0.082514,0.087467


# Preprocessing
- The target variable price is set to have values between 10 and 350, all records with prices outside this range are dropped.
- The last_review columns is formatted to be of datatime format.

In [3]:
# Drop outliers
min_price = 10
max_price = 350
idx = df['price'].between(min_price, max_price)
df = df[idx].copy()
# Convert last_review to datetime
df['last_review'] = pd.to_datetime(df['last_review'])

In [4]:
# Sanity Check
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 46428 entries, 0 to 48894
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   id                              46428 non-null  int64         
 1   name                            46413 non-null  object        
 2   host_id                         46428 non-null  int64         
 3   host_name                       46407 non-null  object        
 4   neighbourhood_group             46428 non-null  object        
 5   neighbourhood                   46428 non-null  object        
 6   latitude                        46428 non-null  float64       
 7   longitude                       46428 non-null  float64       
 8   room_type                       46428 non-null  object        
 9   price                           46428 non-null  int64         
 10  minimum_nights                  46428 non-null  int64         
 11  number_

In [5]:
# Finishing the wandb run so logs and artifact are synchronized.
run.finish()